In [2]:
import pandas as pd
from itables import init_notebook_mode
import datetime
import pickle
import os
import phonenumbers as pn

from api_clients.google_ads_api import GoogleAdsAPIWrapper
from api_clients.facebook_api import get_campaigns
from api_clients import hubspot_api as hubspot
from data_sources.hubspot_conversions import get_calendly_data, get_first_calls

In [3]:
fetch_calendly_data = False

In [4]:
datasets = os.listdir("./data")
datasets

['first_call_analytics_2025-05-05.pkl',
 'first_call_analytics_2025-05-04.pkl',
 'calendly_first_call_data.csv',
 'calendly_first_calls_2024-09-01_2025-05-04.pkl']

In [5]:
if fetch_calendly_data:
    calendly_first_calls_raw = get_calendly_data(
        datetime.datetime.strptime("2024-09-01", "%Y-%m-%d")
    )
    pickle.dump(
        calendly_first_calls_raw,
        open(
            f"./data/calendly_first_calls_2024-09-01_{datetime.datetime.now().strftime('%Y-%m-%d')}.pkl",
            "wb",
        ),
    )
else:
    datasets = os.listdir("./data")
    path = "./data/" + max(
        [d for d in datasets if d.startswith("calendly_first_calls_2024-09-01")],
        key=lambda x: datetime.datetime.strptime(x.split("_")[-1][:-4], "%Y-%m-%d"),
    )
    calendly_first_calls_raw = pickle.load(
        open(
            path,
            "rb",
        )
    )

calendly_first_calls_raw

,created_at,email,name,new_invitee,cancellation,utm_campaign,utm_source,utm_medium,utm_content,utm_term
created_at,,,,,,,,,,
2024-08-27 04:04:09.549747+00:00,2024-08-27T04:04:09.565869Z,ashabdesign@outlook.com,Ashab,None,NaN,None,None,None,None,None
2024-08-27 19:37:43.208148+00:00,2024-08-27T19:37:43.227112Z,alnoman956@gmail.com,ABDULLAH AL NOMAN,None,NaN,None,None,None,None,None
2024-08-29 20:13:25.966348+00:00,2024-08-29T20:13:25.984025Z,noakadim@gmail.com,Noa,https://api.calendly.com/scheduled_events/b474...,"{'canceled_by': 'Noa', 'canceler_type': 'invit...",None,None,None,None,None
2024-08-30 09:25:24.265453+00:00,2024-08-30T09:25:24.277806Z,enocknoutevi89@gmail.com,Enock Noutevi,None,NaN,None,None,None,None,None
2024-08-30 11:12:38.788992+00:00,2024-08-30T11:12:38.807121Z,jenniferanne.amrein@gmail.com,Jennifer Anne Amrein,None,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
2025-05-02 22:41:52.368347+00:00,2025-05-02T22:41:52.391900Z,areeburrehman381@gmail.com,Areeb Ur Rehman,None,NaN,HM | Search | BRAND | Homepage,adwords,ppc,None,lalia berlin
2025-05-03 13:44:08.313635+00:00,2025-05-03T13:44:08.332390Z,melodyiris6@gmail.com,Melody,None,NaN,HM | Pmax | Homepage,adwords,ppc,None,None
2025-05-03 20:24:45.706321+00:00,2025-05-03T20:24:45.720526Z,allert@ukr.net,Iрина Slavetska,None,NaN,HM - TOF - ANGLE TESTING - ABO,ig,Instagram_Feed,[T] TOF - AT - New Copies - American Girl (Video),[T] HM - TOF - AT - Value-Based Purchase LLA -...


In [6]:
calendly_first_calls = calendly_first_calls_raw[
    [
        "email",
        "name",
        "new_invitee",
        "cancellation",
        "utm_campaign",
        "utm_source",
        "utm_medium",
        "utm_content",
        "utm_term",
    ]
]
calendly_first_calls["cancelled"] = (
    calendly_first_calls["cancellation"].notna()
    & calendly_first_calls["new_invitee"].isna()
)
calendly_first_calls["rescheduled"] = (
    calendly_first_calls["cancellation"].notna()
    & calendly_first_calls["new_invitee"].notna()
)
calendly_first_calls.drop(columns=["cancellation", "new_invitee"], inplace=True)
calendly_first_calls.fillna(pd.NA, inplace=True)
calendly_first_calls = (
    calendly_first_calls.reset_index()
    .groupby("email")
    .agg(
        {
            "created_at": "first",
            "name": "first",
            "email": "first",
            "utm_campaign": "first",
            "utm_source": "first",
            "utm_medium": "first",
            "utm_content": "first",
            "utm_term": "first",
            "cancelled": "any",
            "rescheduled": "any",
        }
    )
    .set_index("created_at")
)


def map_calendly_utm(row):
    if row["utm_source"] in ["ig", "fb", "an", "facebook"]:
        return "Facebook", row["utm_campaign"]
    elif row["utm_source"] in ["adwords"]:
        return "Google", row["utm_campaign"]
    else:
        return pd.NA, pd.NA


calendly_first_calls[["source", "campaign"]] = (
    calendly_first_calls[["utm_source", "utm_campaign"]]
    .apply(map_calendly_utm, axis=1)
    .to_list()
)
calendly_first_calls


,name,email,utm_campaign,utm_source,utm_medium,utm_content,utm_term,cancelled,rescheduled,source,campaign
created_at,,,,,,,,,,,
2024-12-22 03:57:58.280269+00:00,Abdirashid Abdi,001abdirashid@gmail.com,general,website,button,None,None,False,False,<NA>,<NA>
2024-11-04 06:06:45.210991+00:00,Caio,123caio@gmail.com,general,website,button,None,None,False,False,<NA>,<NA>
2024-10-26 14:05:51.841803+00:00,Ali,22110243@lums.edu.pk,None,None,None,None,None,False,False,<NA>,<NA>
2024-09-27 09:10:10.261098+00:00,Amit Lau,415.amit@gmail.com,None,None,None,None,None,False,False,<NA>,<NA>
2024-10-27 19:31:58.723619+00:00,Abdulaziz,a.abdelkadr@outlook.com,Retargeting | September 2023 |,fb,Facebook_Mobile_Reels,HM - M/BOF - BP Copy2 - American Girl (Video),Ad set2: WARM | Loft_leads |,False,False,Facebook,Retargeting | September 2023 |
...,...,...,...,...,...,...,...,...,...,...,...
2024-09-24 05:33:00.438041+00:00,Ziv Gitelman,zivgit@gmail.com,None,None,None,None,None,False,False,<NA>,<NA>
2024-11-24 15:30:43.134520+00:00,Lisa,zohrehbouzari@gmail.com,None,None,None,None,None,False,False,<NA>,<NA>
2024-11-05 14:20:05.717237+00:00,ZUBAIR Amini,zubairamini@gmail.com,None,None,None,None,None,False,False,<NA>,<NA>


In [7]:
contact_properties = [
    "fit",
    "email",
    "about_me_short_label",
    "manual_referral_source",
    "open_deals",
    "sale_count_gsheet_sync",
    "hs_analytics_source",
    "hs_analytics_source_data_1",
    "hs_analytics_source_data_2",
    "first_call_notes",
    "phone",
    "level",
    "ip_country",
    "city",
    "goals",
]

lead_properties = ["within_budget_"]


In [8]:
hs_meetings_raw = hubspot.search_objects(
    object_type="meetings",
    filter_groups=[
        {
            "filters": [
                {
                    "propertyName": "hs_meeting_title",
                    "operator": "EQ",
                    "value": "Calendly: First call with LALIA",
                },
            ],
        },
        {
            "filters": [
                {
                    "propertyName": "hs_activity_type",
                    "operator": "EQ",
                    "value": "First Call",
                },
            ],
        },
        {
            "filters": [
                {
                    "propertyName": "hs_meeting_body",
                    "operator": "CONTAINS_TOKEN",
                    "value": "First call",
                },
            ],
        },
    ],
    properties=[
        "hs_meeting_title",
        "hs_activity_type",
        "hs_meeting_start_time",
        "hs_meeting_outcome",
        "hubspot_owner_id",
        "hs_guest_emails",
        "hs_internal_meeting_notes",
    ],
)

In [ ]:
hs_meetings = pd.json_normalize(hs_meetings_raw)[
    [
        "id",
        "properties.hs_activity_type",
        "properties.hs_meeting_outcome",
        "properties.hs_internal_meeting_notes",
        "properties.hubspot_owner_id",
    ]
].rename(columns={"id": "meeting_id"})
hs_meetings

,meeting_id,properties.hs_activity_type,properties.hs_meeting_outcome,properties.hs_internal_meeting_notes,properties.hubspot_owner_id
0,27070610886,First Call,SCHEDULED,None,362782088
1,27072654834,First Call,SCHEDULED,None,None
2,27091118812,First Call,SCHEDULED,None,None
3,27101483452,First Call,SCHEDULED,None,None
4,27131671776,First Call,SCHEDULED,None,None
...,...,...,...,...,...
3648,191812200660,None,SCHEDULED,None,2114235671
3649,191897514188,None,None,None,1305224860
3650,191901156545,None,SCHEDULED,None,1305224860
3651,191855433926,None,None,None,1305224860


In [10]:
hs_list_first_call = hubspot.get_list_memberships(list_id="157")
contact_ids = [m["recordId"] for m in hs_list_first_call]


text/html; charset=UTF-8 <!doctype html><html lang="en"><head>
    <meta charset="utf-8">
    <title>HubSpot Developers 🚀 </title>
    <link rel="shortcut icon" href="https://developers.hubspot.com/hubfs/HubSpot_Logos/HubSpot-Inversed-Favicon.png">
    <meta name="description" content="HubSpot's API, CMS, and UI Extension documentation, developer community, and development resources. ">
    
    <link rel="preconnect" href="https://hubspotusercontent-na1.net">
    <link rel="preconnect" href="//cdn2.hubspot.net">
    <link rel="preload" crossorigin href="https://53.fs1.hubspotusercontent-na1.net/hubfs/53/tools/fonts/Queens-Medium.woff2" as="font">
    <link rel="preload" crossorigin href="https://53.fs1.hubspotusercontent-na1.net/hubfs/53/tools/fonts/LexendDeca-Light.woff2" as="font">
    
    
    
    
    
    <meta name="viewport" content="width=device-width, initial-scale=1">

    
    <meta property="og:description" content="HubSpot's API, CMS, and UI Extension documentation, dev

In [11]:
hs_contacts_raw = hubspot.batch_get_objects(
    object_type="contacts",
    object_ids=contact_ids,
    properties=contact_properties,
)
hs_contacts = pd.json_normalize(hs_contacts_raw).rename(columns={"id": "contact_id"})


In [12]:
google_ads_api = GoogleAdsAPIWrapper()
google_ads_campaigns = pd.json_normalize(google_ads_api.get_campaigns())
facebook_campaigns = get_campaigns(fields=["name", "id"])
facebook_campaigns = pd.DataFrame([dict(c) for c in facebook_campaigns])

In [13]:
google_campaign_names = (
    google_ads_campaigns["campaign.name"]
    .apply(lambda x: x.replace("_", " ").replace(" ", "").strip().lower())
    .to_list()
)
facebook_campaign_names = facebook_campaigns["name"].to_list()


def map_hs_analytics_source(row):
    if row["properties.hs_analytics_source"] == "PAID_SEARCH":
        if (
            row["properties.hs_analytics_source_data_1"]
            .replace("_", " ")
            .replace(" ", "")
            .strip()
            .lower()
            in google_campaign_names
        ):
            return "Google", row["properties.hs_analytics_source_data_1"]
        else:
            return "Google", pd.NA
    elif row["properties.hs_analytics_source"] == "PAID_SOCIAL":
        if row["properties.hs_analytics_source_data_1"] == "Facebook":
            return "Facebook", row["properties.hs_analytics_source_data_2"]

    return pd.NA, pd.NA


hs_contacts[["source", "campaign"]] = (
    hs_contacts[
        [
            "properties.hs_analytics_source",
            "properties.hs_analytics_source_data_1",
            "properties.hs_analytics_source_data_2",
        ]
    ]
    .apply(map_hs_analytics_source, axis=1)
    .to_list()
)

In [14]:
hs_contacts["sale"] = (
    hs_contacts["properties.sale_count_gsheet_sync"].astype("Int64") > 0
).fillna(False)

In [15]:
contacts_to_meetings = hubspot.batch_read_associations(
    from_object_type="contacts",
    to_object_type="meetings",
    object_ids=contact_ids,
)
pd.json_normalize(contacts_to_meetings)
meeting_associations = []
for contact_id, d in contacts_to_meetings.items():
    for meetings in d.values():
        for meeting in meetings:
            meeting_associations.append(
                {
                    "contact_id": contact_id,
                    "meeting_id": str(meeting["toObjectId"]),
                }
            )
meeting_associations_df = pd.DataFrame(meeting_associations)

In [ ]:
hs_data = hs_contacts.merge(meeting_associations_df, on="contact_id", how="left").merge(
    hs_meetings, on="meeting_id", how="left"
)
hs_data


,contact_id,createdAt,updatedAt,archived,properties.about_me_short_label,properties.city,properties.createdate,properties.email,properties.first_call_notes,properties.fit,...,properties.phone,properties.sale_count_gsheet_sync,source,campaign,sale,meeting_id,properties.hs_activity_type,properties.hs_meeting_outcome,properties.hs_internal_meeting_notes,properties.hubspot_owner_id
0,10470192858,2024-04-30T11:32:05.193Z,2025-04-10T09:14:00.064Z,False,i am blessing serwaa 39 years and i am learnin...,None,2024-04-30T11:32:05.193Z,blessingserwaa@gmx.de,None,None,...,+49 1575 8740910,None,<NA>,<NA>,False,27326715842,First Call,SCHEDULED,None,1305224860
1,11881738737,2024-05-08T07:43:45.773Z,2025-05-01T10:07:22.976Z,False,"My name is Rom, I'm studying my MA in Choreogr...",Frankfurt am Main,2024-05-08T07:43:45.773Z,rom868@gmail.com,None,None,...,+491774972038,1,<NA>,<NA>,True,30184282561,None,None,None,642753864
2,11881738737,2024-05-08T07:43:45.773Z,2025-05-01T10:07:22.976Z,False,"My name is Rom, I'm studying my MA in Choreogr...",Frankfurt am Main,2024-05-08T07:43:45.773Z,rom868@gmail.com,None,None,...,+491774972038,1,<NA>,<NA>,True,30184352711,First Call,PLACEMENT SCHEDULED,"<div style="""" dir=""auto"" data-top-level=""true""...",642753864
3,11881738737,2024-05-08T07:43:45.773Z,2025-05-01T10:07:22.976Z,False,"My name is Rom, I'm studying my MA in Choreogr...",Frankfurt am Main,2024-05-08T07:43:45.773Z,rom868@gmail.com,None,None,...,+491774972038,1,<NA>,<NA>,True,30218905844,NaN,NaN,NaN,NaN
4,11881738737,2024-05-08T07:43:45.773Z,2025-05-01T10:07:22.976Z,False,"My name is Rom, I'm studying my MA in Choreogr...",Frankfurt am Main,2024-05-08T07:43:45.773Z,rom868@gmail.com,None,None,...,+491774972038,1,<NA>,<NA>,True,30219045822,First Call,None,None,642753864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902,221491308760,2025-03-09T19:43:45.030Z,2025-04-29T08:38:21.132Z,False,Want to know more,None,2025-03-09T19:43:45.030Z,fuzumoe@gmail.com,None,4,...,+49 1521 7136172,None,<NA>,<NA>,False,157120364764,None,VERBAL AGREEMENT,None,1305224860
3903,221491308760,2025-03-09T19:43:45.030Z,2025-04-29T08:38:21.132Z,False,Want to know more,None,2025-03-09T19:43:45.030Z,fuzumoe@gmail.com,None,4,...,+49 1521 7136172,None,<NA>,<NA>,False,170129335502,First Call,SCHEDULED,"<div style="""" dir=""auto"" data-top-level=""true""...",1305224860
3904,221491308760,2025-03-09T19:43:45.030Z,2025-04-29T08:38:21.132Z,False,Want to know more,None,2025-03-09T19:43:45.030Z,fuzumoe@gmail.com,None,4,...,+49 1521 7136172,None,<NA>,<NA>,False,170229513416,NaN,NaN,NaN,NaN
3905,218538044633,2025-03-05T19:55:31.527Z,2025-04-10T09:14:00.346Z,False,"I am a Ghanaian nurse now living in Germany, I...",None,2025-03-05T19:55:31.527Z,deemarioty80@gmail.com,None,None,...,+49 176 84428648,None,<NA>,<NA>,False,155726657783,None,None,None,2114235671


In [17]:
from phonenumbers import geocoder


def get_phone_number_country(phone_number):
    try:
        parsed_number = pn.parse(phone_number, "DE")
        rc = pn.geocoder.region_code_for_country_code(parsed_number.country_code)
        return pn.geocoder._region_display_name(rc, "en")
    except pn.NumberParseException:
        return None


hs_data["phone_country"] = hs_data["properties.phone"].apply(get_phone_number_country)
hs_data = hs_data.rename(
    columns={
        "properties.about_me_short_label": "about_me",
        "properties.email": "email",
        "properties.city": "city",
        "properties.fit": "fit",
        "properties.first_call_notes": "first_call_notes",
        "properties.hs_analytics_source_data_2": "hs_analytics_source_data_2",
        "properties.ip_country": "ip_country",
        "properties.level": "level",
        "properties.manual_referral_source": "manual_referral_source",
        "properties.open_deals": "open_deals",
        "properties.sale_count_gsheet_sync": "sales",
        "properties.hs_meeting_outcome": "meeting_outcome",
        "properties.hs_internal_meeting_notes": "meeting_notes",
        "createdAt": "date",
    }
)

In [18]:
hs_data.first_call_notes.fillna("", inplace=True)
hs_data = (
    hs_data[
        [
            "date",
            "email",
            "about_me",
            "city",
            "fit",
            "first_call_notes",
            "source",
            "campaign",
            "phone_country",
            "level",
            "sale",
            "meeting_outcome",
        ]
    ]
    .groupby("email")
    .agg(
        {
            "date": "first",
            "about_me": "first",
            "city": "first",
            "fit": "first",
            "first_call_notes": " ".join,
            "source": "first",
            "campaign": "first",
            "phone_country": "first",
            "level": "first",
            "sale": "any",
            "meeting_outcome": "last",
        }
    )
)


/tmp/ipykernel_15391/3730914958.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hs_data.first_call_notes.fillna("", inplace=True)


In [23]:
variables = [
    "about_me",
    "city",
    "fit",
    "first_call_notes",
    "source",
    "campaign",
    "phone_country",
    "level",
    "sale",
    "rescheduled",
    "cancelled",
    "date",
    "meeting_outcome",
]
hs_data = hs_data.reset_index().fillna(pd.NA)

data = hs_data.merge(
    calendly_first_calls.reset_index()[
        ["created_at", "email", "rescheduled", "cancelled", "source", "campaign"]
    ],
    on="email",
    how="left",
)

data["campaign"] = data["campaign_y"].fillna(data["campaign_x"])
data["source"] = data["source_y"].fillna(data["source_x"])


data = data[variables]

In [24]:
data["about_me_wc"] = data["about_me"].apply(lambda x: len(str(x).split(" ")))
data

,about_me,city,fit,first_call_notes,source,campaign,phone_country,level,sale,rescheduled,cancelled,date,meeting_outcome,about_me_wc
0,Hi my name is Abdirashid i am from somalia i a...,<NA>,<NA>,,<NA>,<NA>,Germany,<NA>,False,False,False,2024-12-22T03:57:59.956Z,NO_SHOW,54
1,"Hi, I live in Berlin for a while now . I´m app...",<NA>,4,,<NA>,<NA>,Germany,<NA>,True,False,False,2024-11-04T06:06:46.880Z,COMPLETED,41
2,My name is Ali. I moved to germany few months ...,<NA>,<NA>,,<NA>,<NA>,Germany,<NA>,False,False,False,2024-10-26T14:05:53.666Z,COMPLETED,66
3,Just duolingo,<NA>,<NA>,,<NA>,<NA>,United States,<NA>,False,False,False,2024-09-27T09:10:11.845Z,COMPLETED,2
4,I want to learn the German language as fast as...,<NA>,<NA>,,Facebook,Retargeting | September 2023 |,Austria,<NA>,False,False,False,2024-10-27T19:32:00.760Z,COMPLETED,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,<NA>,<NA>,<NA>,,<NA>,<NA>,<NA>,<NA>,False,NaN,NaN,2024-05-26T09:51:11.786Z,COMPLETED,1
1318,I would love to learn German language and i be...,<NA>,<NA>,,<NA>,<NA>,Germany,<NA>,False,False,False,2024-11-24T15:30:44.918Z,COMPLETED,18
1319,"Hallo, guten Abend. Ich bin 22 Jahre alt, lebe...",<NA>,<NA>,,<NA>,<NA>,Germany,<NA>,False,False,False,2024-11-05T14:20:09.086Z,COMPLETED,20
1320,So far I completed A1 and A2.1 classes from DW...,Berlin,<NA>,,<NA>,<NA>,Germany,IL2,True,False,False,2024-12-05T19:29:23.686Z,COMPLETED,36


In [ ]:
data.source = data.source.astype("category")
data.campaign = data.campaign.astype("category")
data.phone_country = data.phone_country.astype("category")
data.level = data.level.astype("category")
data.meeting_outcome = data.meeting_outcome.astype("category")
data.fit = data.fit.astype("category")
# data.date = data.date.astype("datetime64[ns]")
data.to_pickle(
    f"data/first_call_analytics_{datetime.datetime.now().strftime('%Y-%m-%d')}.pkl"
)
data.dtypes

In [30]:
from ydata_profiling import ProfileReport

profile = ProfileReport(data, title="Profiling Report")
profile.to_notebook_iframe()

[2025-05-05 11:23:06 - INFO] Pandas backend loaded 2.2.3
[2025-05-05 11:23:06 - INFO] Numpy backend loaded 2.1.3
[2025-05-05 11:23:06 - INFO] Pyspark backend NOT loaded
[2025-05-05 11:23:06 - INFO] Python backend loaded


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 14/14 [00:00<00:00, 442.16it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]